In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-07-17T18:14:49.851490-04:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.10.0

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 5.15.0-76-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Quadro RTX 5000



In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import sklearn
import gc
import os

In [5]:
%watermark --iversions

numpy  : 1.25.1
sklearn: 1.3.0
pandas : 2.0.3



In [6]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [7]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')
sample_submission = pd.read_csv('../input/sample_submission.csv.zip')

In [8]:
columns = test.columns[1:]

In [9]:
X = train[columns].values
X_test = test[columns].values
Y = train.target.values

In [57]:
%%time
train_oof = np.zeros((X.shape[0], ))
test_preds = 0
train_oof.shape

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(X)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target = Y[train_index]
    
    val_features = X[val_index]
    val_target = Y[val_index]
    
    model = HistGradientBoostingClassifier(max_iter=1800, learning_rate = 0.005, validation_fraction=None, 
                                           max_depth=10, random_state=42)
    model.fit(train_features, train_target)
    val_pred = model.predict_proba(val_features)[:,1]
    train_oof[val_index] = val_pred
    print("Fold normalized:", gini_normalized(val_target, val_pred))
    test_preds += model.predict_proba(X_test)[:,1]/n_splits
    del train_features, train_target, val_features, val_target
    gc.collect()

Fitting fold 1
Fold normalized: 0.2826404904629799
Fitting fold 2
Fold normalized: 0.2900991098434515
Fitting fold 3
Fold normalized: 0.27227223692921887
Fitting fold 4
Fold normalized: 0.2795800880508094
Fitting fold 5
Fold normalized: 0.273442372028705
CPU times: user 40min 39s, sys: 1.8 s, total: 40min 41s
Wall time: 5min 7s


In [58]:
gini_normalized(Y, train_oof)

0.27938769785394113

In [59]:
roc_auc_score(Y, train_oof)

0.6396938489269706

In [60]:
sample_submission['target'] = test_preds

In [62]:
sample_submission.to_csv('../submissions/HGBC_5_fold.csv.zip', index=False, compression='zip')

On the leaderbaord this submission scores 0.2787 on the public test set, and 0.28261 on the private dataset.